<a href="https://colab.research.google.com/github/MAHA06/ThesisOulu/blob/master/Adversarials/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# from google.colab import drive
# drive.mount(‘/content/gdrive’)

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets
np.random.seed(42) 
torch.manual_seed(42)
import pickle
import matplotlib.pyplot as plt


In [49]:
train_batch,test_batch=128,128
img_size=(28,28)
num_classes=10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.0,), (1.0,))])
dataset = datasets.MNIST(root = './data', train=True, transform = transform, download=True)
train_set, val_set = torch.utils.data.random_split(dataset, [50000, 10000])
test_set = datasets.MNIST(root = './data', train=False, transform = transform, download=True)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=1,shuffle=True) 
val_loader = torch.utils.data.DataLoader(val_set,batch_size=1,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=1,shuffle=True)

In [50]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

In [51]:
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using the device:",device)

Using the device: cuda


In [52]:
def fitModel(model,optimizer,scheduler,criterion,device,train_loader,val_loader,epochs):
  data_loader={'train':train_loader,'val':val_loader}
  print("Starting to fit the model")
  train_error,test_error=[],[]
  best_points=torch.zeros(img_size+(num_classes,))
  best_scores=torch.zeros(num_classes)
  print(best_scores[1])
  for epc in range(epochs):
    loss_epoch,val_loss_epoch=0,0
    for phase in ('train','val'):
      for i,data in enumerate(data_loader[phase]):
        input,label=data[0].to(device),data[1].to(device)
        out=model(input)
        loss=criterion(out,label)
        predict_label=torch.argmax(out,dim=1)
        out=out.squeeze().cpu()
        if phase == 'train':
          optimizer.zero_grad()

          loss.backward()
          optimizer.step()
          loss_epoch+=loss.item()
          print(predict_label.item(),label.item())
          print(out)
          if epc==epochs-1 and predict_label.item()==label.item() and out[predict_label]>best_scores[predict_label.item()]:#if last epoch and the predicted label is correct
            best_scores[predict_label.item()]=out[predict_label.item()]
            best_points[:,:,predict_label]=input.squeeze().detach().cpu().numpy()
            print(best_points[:,:,label.item()].shape)
            
        else:
          val_loss_epoch+=loss.item()
    scheduler.step(val_loss_epoch/len(val_loader))
    print("Epoch : {} Train Loss : {} Eval Loss : {}".format(epc+1,loss_epoch/len(train_loader),val_loss_epoch/len(val_loader)))
    train_error.append(loss_epoch)
    test_error.append(val_loss_epoch)
  
  return train_error,test_error,best_points,best_scores
        
          




In [53]:
model=Net().to(device)

optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
criterion = nn.NLLLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


###############


In [54]:
## RUN THE TRAINING
loss,val_loss,best_points,best_scores=fitModel(model,optimizer,scheduler,criterion,device,train_loader,val_loader,1)


Starting to fit the model
tensor(0.)
4 2
4 5
2 5
5 0
4 6
5 1
5 9
5 9
5 0
5 8
5 6
1 3
5 6
5 5
9 8
5 8
5 6
5 1
5 6
5 1
5 1
6 0
5 4
5 1
5 8
5 0
4 3
6 3
5 4
6 9
5 9
1 1
6 5
8 9
4 0
8 8
1 8
1 8
1 7
3 2
1 6
3 4
5 7
6 2
1 7
4 7
6 9
8 4
3 0
8 2
4 1
1 6
8 4
1 7
8 8
8 5
1 1
4 5
4 7
1 1
8 3
4 6
8 2
8 3
5 4
5 4
4 2
9 4
4 7
3 3
8 7
8 4
3 3
4 5
3 4
5 9
4 8
5 8
8 7
1 1
5 6
4 7
4 2
4 0
4 8
4 3
5 3
8 8
4 3
6 3
5 7
1 7
3 6
4 7
4 9
4 0
3 8
3 9
8 8
5 8
9 7
3 9
3 1
4 4
3 2
3 0
3 0
3 4
3 8
8 2
8 5
3 8
3 7
3 7
1 1
5 6
3 7
3 3
3 9
1 7
3 1
3 7
8 6
6 9
3 6
3 2
3 5
9 9
1 1
9 9
3 3
3 7
1 8
1 1
3 2
3 3
5 7
8 8
1 8
4 9
8 5
1 1
1 7
8 4
3 2
3 6
3 6
9 6
3 0
1 9
4 9
9 5
4 7
3 4
3 2
9 0
1 1
9 7
9 7
4 7
3 9
9 3
9 1
3 4
9 0
3 0
7 4
9 5
9 9
9 8
1 1
3 5
3 3
8 8
9 4
1 1
9 8
9 4
5 5
7 7
9 7
9 6
8 0
9 4
7 9
8 1
5 9
5 2
4 9
5 0
5 2
9 3
8 2
1 2
9 5
1 2
4 4
4 5
1 8
0 9
1 1
5 3
5 7
5 2
5 3
8 8
0 0
5 4
0 6
3 7
1 3
3 5
4 6
2 7
8 0
8 6
6 3
0 4
7 2
4 7
1 1
1 2
1 1
3 2
6 0
4 4
8 9
5 2
1 7
2 0
4 4
1 2
5 9
2 6
4 8
4 7
5 7
4 7
1 9
0 8
5 8

KeyboardInterrupt: ignored

In [ ]:
print(best_points.shape)
plt.imshow(best_points[:,:,2])
print(best_scores)

In [ ]:
filename = 'model_MNIST'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
def deepfool(image, net, num_classes, overshoot, max_iter):

    """
       :param image:
       :param net: network (input: images, output: values of activation **BEFORE** softmax).
       :param num_classes: num_classes (limits the number of classes to test against, by default = 10)
       :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
       :param max_iter: maximum number of iterations for deepfool (default = 50)
       :return: minimal perturbation that fools the classifier, number of iterations that it required, new estimated_label and perturbed image
    """

    is_cuda = torch.cuda.is_available()
    if is_cuda:
        image = image.cuda()
        net = net.cuda()

    f_image = net.forward(Variable(image[None, :, :, :], requires_grad=True)).data.cpu().numpy().flatten()
    I = f_image.argsort()[::-1]

    I = I[0:num_classes]
    label = I[0]

    input_shape = image.cpu().numpy().shape
    pert_image = copy.deepcopy(image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0

    x = Variable(pert_image[None, :], requires_grad=True)
    fs = net.forward(x)
    k_i = label

    while k_i == label and loop_i < max_iter:

        pert = np.inf
        fs[0, I[0]].backward(retain_graph=True)
        grad_orig = x.grad.data.cpu().numpy().copy()

        for k in range(1, num_classes):
            zero_gradients(x)

            fs[0, I[k]].backward(retain_graph=True)
            cur_grad = x.grad.data.cpu().numpy().copy()

            # set new w_k and new f_k
            w_k = cur_grad - grad_orig
            f_k = (fs[0, I[k]] - fs[0, I[0]]).data.cpu().numpy()

            pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())

            # determine which w_k to use
            if pert_k < pert:
                pert = pert_k
                w = w_k

        # compute r_i and r_tot
        # Added 1e-4 for numerical stability
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(r_tot + r_i)

        if is_cuda:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()
        else:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot)

        x = Variable(pert_image, requires_grad=True)
       # print(image.shape)
       # print(x.view(1,1,image.shape[0],-1).shape)
        fs = net.forward(x.view(1,1,image.shape[1],-1))
        k_i = np.argmax(fs.data.cpu().numpy().flatten())

        loop_i += 1

    return (1+overshoot)*r_tot, loop_i, label, k_i, pert_image
def fgsm_attack(input,epsilon,data_grad):
  pert_out = input + epsilon*data_grad.sign()
  pert_out = torch.clamp(pert_out, 0, 1)
  return pert_out

def ifgsm_attack(input,epsilon,data_grad):
  iter = 10
  alpha = epsilon/iter
  pert_out = input
  for i in range(iter-1):
    pert_out = pert_out + alpha*data_grad.sign()
    pert_out = torch.clamp(pert_out, 0, 1)
    if torch.norm((pert_out-input),p=float('inf')) > epsilon:
      break
  return pert_out

def mifgsm_attack(input,epsilon,data_grad):
  iter=10
  decay_factor=1.0
  pert_out = input
  alpha = epsilon/iter
  g=0
  for i in range(iter-1):
    g = decay_factor*g + data_grad/torch.norm(data_grad,p=1)
    pert_out = pert_out + alpha*torch.sign(g)
    pert_out = torch.clamp(pert_out, 0, 1)
    if torch.norm((pert_out-input),p=float('inf')) > epsilon:
      break
  return pert_out

In [ ]:
def JacobianMatrix(model,input):
  print("Input size ",input.shape)
  number_of_piels=input.shape[0]*input.shape[1]
  J = torch.zeros ((1, 784, 10))   # loop will fill in Jacobian
  input.requires_grad = True
  preds = model (input)
  for  i in range (10):
      grd = torch.zeros ((1, 10))   # same shape as preds
      grd[0, i] = 1    # column of Jacobian to compute
      preds.backward (gradient = grd, retain_graph = True)
      J[:,:,i] = input.grad   # fill in one column of Jacobian
      input.grad.zero_()   # .backward() accumulates gradients, so reset to zero
def JSM_attack(model, input):
  input.requires_grad=True
  jacobians

In [ ]:
filename = './Models/model_MNIST_cpu'

model= pickle.load(open(filename, 'rb'))
def testModel(model,criterion,device,test_loader,attack):
  for data,labels in test_loader:
    data,labels=data.to(device),labels.to(device)
    data.requires_grad=True
    outs=model(data)
    true_preds=outs.argmax(outs,dim=1)
    if true_preds!=labels:
      continue
    epsilon=0.1
    data_grad=data.grad.data
    true_loss=criterion(outs,labels)
    if attack == "fgsm":
      perturbed_data = fgsm_attack(data,epsilon,data_grad)
    elif attack == "ifgsm":
      perturbed_data = ifgsm_attack(data,epsilon,data_grad)
    elif attack == "mifgsm":
      perturbed_data = mifgsm_attack(data,epsilon,data_grad)
    elif attack == "jacobian":
      print("Implement the jacobian")
    p_out=model(perturbed_data)
    # p_prediction=p_out.
testModel(model,criterion,'cpu',test_loader,"jacobian")